In [1]:
import os

import pandas as pd
import numpy as np

file_dir = os.getcwd()
file = r"four_grp1_raw.xlsx"

data = os.path.join(file_dir, file)
df = pd.read_excel(data)

In [2]:
df = df[["Trajectory", "Average_Brightness", "Length (frames)", "MSD"]]
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True).sort_values(by="Trajectory")

In [3]:
def keep_these(df, *, brightness: tuple = (3.1, 3.8), min_length: int = 10, diffusion: tuple = (0.3, 3.5)):
    assert min_length < np.max(df["Length (frames)"])
    keepers= df.loc[((df["Average_Brightness"].between(brightness[0], brightness[1])) & (df["Length (frames)"] > min_length ) & (df["MSD"].between(diffusion[0], diffusion[1])))].assign(Keep=1)
    return pd.merge(df, keepers, how="left").fillna(0)

In [4]:
def normalize(df: pd.DataFrame):
    for col in df:
        mean = np.mean(df[col])
        std = np.std(df[col])
        df[col] = (df[col]-mean)/std
        print(f"{col=}, {mean=}, {std=}")
    return df

In [5]:
df = keep_these(df)
parameters = df.iloc[:, 1:-1]
norm_params = normalize(parameters)

identities = df.iloc[:, -1]

array = df.to_numpy()
inputs = parameters.to_numpy()
outputs = identities.to_numpy()

col='Average_Brightness', mean=3.262539406499331, std=0.6068537771669426
col='Length (frames)', mean=28.294966561070044, std=51.83793694244665
col='MSD', mean=6.48642633164615, std=12.98552276557708


In [6]:
indeces_permutation = np.random.permutation(len(array))
shuffled_inputs = inputs[indeces_permutation]
shuffled_targets = outputs[indeces_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

NameError: name 'array' is not defined

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([layers.Dense(64, activation="relu"),layers.Dense(64, activation="relu"),layers.Dense(1, activation="sigmoid")])